In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/train/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/test/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [4]:
efficientnetb3 = tf.keras.applications.efficientnet.EfficientNetB1(include_top=False, weights=None,input_shape=(128, 128, 3), pooling='avg')
x4 = tf.keras.layers.Dense(1, activation='sigmoid')(efficientnetb3.output)
efficientnetb3_model = tf.keras.Model(efficientnetb3.input, x4)

In [5]:
METRICS = [
        'accuracy',
    ]
    
efficientnetb3_model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=METRICS
    )

In [6]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [7]:
history = efficientnetb3_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[lr_scheduler]
)

Epoch 1/10


2023-05-02 18:53:27.450430: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


652/652 [==============================] - 171s 179ms/step - loss: 0.4660 - accuracy: 0.8023 - val_loss: 2.8033 - val_accuracy: 0.6250 - lr: 0.0100
Epoch 2/10
652/652 [==============================] - 84s 128ms/step - loss: 0.2860 - accuracy: 0.8827 - val_loss: 0.3938 - val_accuracy: 0.8478 - lr: 0.0089
Epoch 3/10
652/652 [==============================] - 83s 128ms/step - loss: 0.2480 - accuracy: 0.8986 - val_loss: 0.6068 - val_accuracy: 0.8109 - lr: 0.0079
Epoch 4/10
652/652 [==============================] - 84s 128ms/step - loss: 0.2023 - accuracy: 0.9239 - val_loss: 1.2061 - val_accuracy: 0.7676 - lr: 0.0071
Epoch 5/10
652/652 [==============================] - 84s 128ms/step - loss: 0.1820 - accuracy: 0.9342 - val_loss: 0.4013 - val_accuracy: 0.8141 - lr: 0.0063
Epoch 6/10
652/652 [==============================] - 83s 127ms/step - loss: 0.1665 - accuracy: 0.9387 - val_loss: 0.3649 - val_accuracy: 0.9038 - lr: 0.0056
Epoch 7/10
652/652 [==============================] - 83s 127m

In [8]:
efficientnetb3_model.save("efficientnet_xray_Pneumonia_b1.h5")